In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, classification_report, accuracy_score, mean_squared_error
from scipy.stats import spearmanr
import os
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. SETUP & DATA LOADING
# ==========================================
print("[INFO] Initializing model training pipeline...")
file_path = r'C:\Users\arnav\OneDrive\Desktop\customer-risk-agent\data\processed\labeled_customers.csv'

# Fallback path logic
if not os.path.exists(file_path):
    file_path = os.path.join(os.getcwd(), '..', 'data', 'processed', 'labeled_customers.csv')

df = pd.read_csv(file_path)
if 'Unnamed: 0' in df.columns: df = df.drop(columns=['Unnamed: 0'])

# Define strict feature set (matches segmentation phase)
features = ['Recency', 'Frequency', 'Avg_Monthly_Spend', 'Tenure', 'AOV', 'Velocity_Recent', 'Velocity_Drift', 'Return_Count']
X = df[features]
y_class = df['Target_Label']
y_reg = df['FutureSales_Label']

# Train/Test Split (80/20)
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(
    X, y_class, y_reg, test_size=0.2, random_state=42
)

print(f"[INFO] Training Data: {len(X_train)} rows | Test Data: {len(X_test)} rows")

# ==========================================
# 2. CLASSIFICATION MODEL (Risk Segmentation)
# ==========================================
print("\n[INFO] Training XGBoost Classifier...")
clf = xgb.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
clf.fit(X_train, y_class_train)

# Evaluation
y_pred_class = clf.predict(X_test)
accuracy = accuracy_score(y_class_test, y_pred_class)

print("\n" + "="*60)
print(f"🛡️  CLASSIFICATION PERFORMANCE (Behavioral Segments)")
print("="*60)
print(f"Global Accuracy: {accuracy:.2%}")
print("-" * 60)
print(classification_report(y_class_test, y_pred_class, target_names=['At-Risk', 'Average', 'High Value']))

# ==========================================
# 3. REGRESSION MODEL (Spend Prediction)
# ==========================================
print("\n[INFO] Training XGBoost Regressor...")
# Log-transform target to handle skewed spend data
y_reg_train_log = np.log1p(y_reg_train.clip(lower=0))

reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, max_depth=5, learning_rate=0.03, random_state=42)
reg.fit(X_train, y_reg_train_log)

# Evaluation
preds_log = reg.predict(X_test)
preds_real = np.expm1(preds_log) # Inverse transform

# Metrics
mae = mean_absolute_error(y_reg_test, preds_real)
rmse = np.sqrt(mean_squared_error(y_reg_test, preds_real))
r2 = r2_score(y_reg_test, preds_real)
rank_corr, _ = spearmanr(y_reg_test, preds_real)

# Baseline Comparison (Mean Predictor)
baseline_mae = mean_absolute_error(y_reg_test, [y_reg_train.mean()] * len(y_reg_test))
improvement = (1 - (mae / baseline_mae)) * 100

print("\n" + "="*60)
print(f"💰 REGRESSION PERFORMANCE (90-Day Spend)")
print("="*60)
print(f"{'Metric':<25} | {'Value':<15} | {'Target / Note':<20}")
print("-" * 60)
print(f"{'Ranking (Spearman)':<25} | {rank_corr:.4f}          | {'> 0.50 (Good)'}")
print(f"{'Avg Error (MAE)':<25} | ${mae:.2f}         | {'Lower is better'}")
print(f"{'RMSE':<25} | ${rmse:.2f}         | {'Penalizes outliers'}")
print(f"{'R² Score':<25} | {r2:.4f}          | {'Variance explained'}")
print("-" * 60)
print(f"Baseline MAE (Mean):      ${baseline_mae:.2f}")
print(f"Model Improvement:        {improvement:.1f}% vs Baseline")

# ==========================================
# 4. EXPORT
# ==========================================
model_dir = r'C:\Users\arnav\OneDrive\Desktop\customer-risk-agent\models'
if not os.path.exists(model_dir): os.makedirs(model_dir)

clf.save_model(os.path.join(model_dir, 'xgb_classifier.json'))
reg.save_model(os.path.join(model_dir, 'xgb_regressor.json'))

print("\n[INFO] Models serialized and saved to:", model_dir)

Loading Data...
Training Classifier...
--- Segment Classification Report ---
              precision    recall  f1-score   support

     At-Risk       0.98      0.97      0.97       274
     Average       0.94      0.97      0.96       289
  High Value       0.98      0.91      0.94       111

    accuracy                           0.96       674
   macro avg       0.97      0.95      0.96       674
weighted avg       0.96      0.96      0.96       674

Training Regressor...
--- Spend Model Results ---
MAE: $909.00
Spearman Rank Correlation: 0.51
✅ Phase 2 Complete: Models Saved.
